In [1]:
library(SingleCellExperiment)
library(devtools)
library(scAnnoX)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats

Warning message:
"package 'matrixStats' was built under R version 4.3.2"

Attaching package: 'MatrixGenerics'


The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, 

In [2]:
# A preprocessed dataset of type Seurat.
test.obj <- readRDS('../data/GSE81252.test.obj')
ref.obj <- readRDS('../data/GSE81252.ref.obj')

### Only one single cell annotation tool is invoked

In [5]:
# Using SingleR as an example.
pred.obj <- autoAnnoTools(
    test.obj,
    ref.obj = ref.obj,
    ref.ctype = 'CellType',
    marker.lst = marker.lst,
    method = 'SingleR',
    select.marker = 'Seurat',
    top.k = 30,
    strategy = 'refernce-based'
) 
head(pred.obj)

,orig.ident,nCount_RNA,nFeature_RNA,cell_id,experiment,assignment_LB,CellType,SingleR
,<fct>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
A1_he2,A1,29630.19,7229,A1_he2,he2,HE,HE,HE
A1_huvec,A1,30603.64,7374,A1_huvec,huvec,EC,EC,EC
A10_he1,A10,26921.42,6185,A10_he1,he1,HE,HE,HE
A10_lb2,A10,21383.21,4296,A10_lb2,lb2,EC-LB,EC-LB,EC-LB
A10_lb5,A10,22637.05,5307,A10_lb5,lb5,MC-LB,MC-LB,MC-LB
A11_he1,A11,16551.88,3725,A11_he1,he1,HE,HE,HE-LB
A11_lb5,A11,24529.42,5102,A11_lb5,lb5,EC-LB,EC-LB,EC-LB
A12_he1,A12,24989.38,5695,A12_he1,he1,HE,HE,HE
A12_huvec,A12,32676.21,7801,A12_huvec,huvec,EC,EC,EC


In [9]:
?findMarkerToolsForSc

In [10]:
# Using SCINA as an example.
Idents(ref.obj) <- ref.obj$CellType
marker.lst <- findMarkerToolsForSc(ref.obj, to.list = TRUE, top.k = 30)
pred.obj <- autoAnnoTools(
    test.obj,
    ref.obj = ref.obj,
    ref.ctype = 'CellType',
    marker.lst = marker.lst,
    method = 'SCINA',
    select.marker = 'Seurat',
    top.k = 30,
    strategy = 'marker-based'
) 
head(pred.obj)

ERROR: Error in findMarkerToolsForSc(ref.obj, to.list = TRUE, top.k = 30): could not find function "findMarkerToolsForSc"


In [ ]:
a <- autoAnnoResult(pred.obj)
a

Warning message:
"The following arguments are not used: drop"


### Multiple single-cell annotation tools are invoked

In [2]:
devtools::load_all('../../scAnnoX')

ℹ Loading scAnnoX
Warning message:
"replacing previous import 'data.table::shift' by 'tictoc::shift' when loading 'CelliD'"


In [3]:
document()

ℹ Updating scAnnoX documentation
ℹ Loading scAnnoX


In [6]:
findMarkerToolsForSc(e)

ERROR: Error in eval(expr, envir, enclos): object 'e' not found


In [4]:
listToolMethods()


$SingleR
function(obj.seu, ref.obj, ref.ctype = NULL, ...) {
    test.expr <- GetAssayData(obj.seu, slot = "data") %>% as.matrix
	ref.expr <- GetAssayData(ref.obj, slot = "data") %>% as.matrix
	
	if (!is.null(ref.ctype)) label.ref <- ref.obj@meta.data[, ref.ctype]
	else label.ref <- Idents(ref.obj)
    res.pred <- SingleR::SingleR(test = test.expr, ref = ref.expr, labels = label.ref, ...)
    obj.seu$SingleR <- res.pred$labels
    return(obj.seu)
}
<environment: namespace:scAnnoX>

$Seurat
function(obj.seu, ref.obj, ref.ctype = NULL, npcs = 30, ...) { 
	 anchors <- FindTransferAnchors(reference = ref.obj, query = obj.seu, dims = 1 : npcs)
     if (!is.null(ref.ctype)) label.ref <- ref.obj@meta.data[, ref.ctype]
     else label.ref <- Idents(ref.obj)
     res.pred <- TransferData(anchorset = anchors, refdata = label.ref,dims = 1 : npcs, ...)
     obj.seu$SeuratAnno <- res.pred[, 1]
     return(obj.seu)
}
<environment: namespace:scAnnoX>

$sciBet
function(obj.seu, ref.obj, ref.ctype = NULL, ...) {
	test.df <- GetAssayData(obj.seu, slot = "data") %>% as.data.frame %>% t
    	ref.df <- GetAssayData(ref.obj, slot = "data") %>% as.data.frame %>% t
    	if (!is.null(ref.ctype)) label.ref <- ref.obj@meta.data[, ref.ctype]
    	else label.ref <- Idents(ref.obj)
	ref.df <- cbind.data.frame(ref.df, label = label.ref %>% as.vector)
	res.pred <- scibetR::SciBet_R(ref.df, test.df, ...)
	obj.seu$sciBet <- res.pred
	return(obj.seu)
}
<environment: namespace:scAnnoX>

$scmap
function(obj.seu, ref.obj, ref.ctype = NULL, slot.data = c('data', 'count'), ...) {
    system.file('modules', 'scmap-master', package = 'scAnnoX') %>% load_all(.)
	
    ref.expr <- GetAssayData(ref.obj, slot = slot.data) %>% as.matrix
    if (!is.null(ref.ctype)) label.ref <- ref.obj@meta.data[, ref.ctype]
	else label.ref <- Idents(ref.obj)
	sce <- SingleCellExperiment(assays = list(normcounts = as.matrix(ref.expr)), colData = label.ref)
    
	if (match.arg(slot.data) == 'count') logcounts(sce) <- log2(normcounts(sce) + 1)
	else logcounts(sce) <- normcounts(sce)
	rowData(sce)$feature_symbol <- rownames(sce)
    sce <- sce[!duplicated(rownames(sce)), ] %>% selectFeatures(.) %>% indexCell(.)
 
    test.expr <- GetAssayData(obj.seu, slot = slot.data) %>% as.matrix
    sce.test <- SingleCellExperiment(assays = list(normcounts = as.matrix(test.expr)))
    if (match.arg(slot.data) == 'count') logcounts(sce.test) <- log2(normcounts(sce.test) + 1)
	else logcounts(sce.test) <- normcounts(sce.test)
    rowData(sce.test)$feature_symbol <- rownames(sce.test)
    sce.test <- sce.test[!duplicated(rownames(sce.test)), ] %>% selectFeatures(.) %>% indexCell(.)

    pre.res <- scmapCell(sce.test, list(metadata(sce)$scmap_cell_index))
    scmap.clusters <- scmapCell2Cluster(pre.res, list(as.character(colData(sce)[, 1])))
    obj.seu$scmapAnno <- scmap.clusters$combined_labs
    return(obj.seu)
}
<environment: namespace:scAnnoX>

$CHETAH
function(obj.seu, ref.obj, ref.ctype = NULL, slot.data = 'data', ...) {
	test.expr <- GetAssayData(obj.seu, slot = slot.data) %>% as.matrix
	sce.test <- SingleCellExperiment(assays = list(counts = test.expr))
	if (slot.data == 'count') sce.test <- scater::logNormCounts(sce.test)

	ref.expr <- GetAssayData(ref.obj, slot = slot.data) %>% as.matrix
	if (!is.null(ref.ctype)) label.ref <- ref.obj@meta.data[, ref.ctype]
	else label.ref <- Idents(ref.obj)
	sce.ref <- SingleCellExperiment(
		assays = list(counts = as.matrix(ref.expr)),
		colData = DataFrame(celltypes = label.ref)
	)
	if (slot.data == 'count') sce.ref <- scater::logNormCounts(sce.ref)
	pred.res <- CHETAH::CHETAHclassifier(input = sce.test, ref_cells = sce.ref, ...)
	obj.seu$CHETAH <- pred.res$celltype_CHETAH %>% data.frame
	return(obj.seu)
}
<environment: namespace:scAnnoX>

$scSorter
function(obj.seu, marker.lst, ...) {
    test.expr <- GetAssayData(obj.seu, slot = "data") %>% as.matrix
    anno <- collapseLisToFrame(marker.lst)
	pred.res <- scSorter::scSorter(expr = test.expr, anno = anno, ...)
	ob